# 소스코드 통합예정

In [ ]:
import pandas as pd
from os.path import join
from os import getcwd

In [ ]:
cur_path = getcwd()
data_path = join(cur_path,'data')
static_path = join(cur_path,'static')

df_test = pd.read_csv(join(data_path,'test.csv'))
df_train = pd.read_csv(join(data_path,'train.csv'))

# 확인용, 나중에 지움
print('[current Absolute Path]\t{}'.format(cur_path))
print('[data folder Absolute Path]\t {}'.format(data_path))
print('[image folder Absolute Path]\t {}'.format(static_path))

In [ ]:
df_test.head(3)

In [ ]:
df_train.head(3)